In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

pd.options.display.max_columns = None

In [29]:
datos = ['https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=60',
        'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=120', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=180',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=240', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=300',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=360', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=420']

In [30]:
nombre = pd.DataFrame()

for i in datos:
    url = i
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    html.status_code
    
    productos = soup.find_all("td", {"class": "col-name"})
    
    jugadores = {'jugador': []}
    
    for x in productos:
        jugadores['jugador'].append(x.find('a').get('href')) 
    
    jugadores_top = pd.DataFrame(jugadores)
    
    jugadores_top['jug'] = jugadores_top['jugador'].str.extract(r'(?<=\/player\/)(.*)(?=\/220057)')
    
    jugadores_fichar = jugadores_top.dropna()
    jugadores_fichar = jugadores_fichar.reset_index(drop=True)
    
    equipos = jugadores_fichar.drop(['jugador'], axis=1)
    
    nombre = pd.concat([nombre, equipos], axis=0)

In [31]:
nombre = nombre.reset_index(drop=True)

In [35]:
nombre.head(2)

,jug
0,266797/gaston-alvarez
1,264432/abdessamad-ezzalzouli


In [36]:
jugar_caract = pd.DataFrame()

for i in range(0,426):
    jug = nombre['jug'][i]
    
    
    url2 = 'https://sofifa.com/player/' + jug + '/220057/'
    html2 = requests.get(url2)
    soup2 = BeautifulSoup(html2.content, "html.parser")
    html2.status_code
    
    productos2 = soup2.find_all("ul", {"class": "pl"})
    
    
    caract = {'jugador': []}
    for i in productos2:
        caract['jugador'].append(i) 
    
    caract = pd.DataFrame(caract)

    caract.rename(columns = {'jugador': jug}, inplace = True)
    caract[jug] = caract[jug].astype(str)
    
    caract[jug] = caract[jug].str.extract(r'(?<=\d\"\>)(\d+)(?=\<\/span\>)')
    
    caract_jug = caract.dropna()
    caract_jug = caract_jug.T

    
    jugar_caract = pd.concat([jugar_caract, caract_jug], axis=0)

In [37]:
jugar_caract = jugar_caract.reset_index(drop=True)

In [48]:
jugar_caract = jugar_caract.drop(jugar_caract.index[[31,80,89,95,116,158,208,263]])

In [50]:
jugar_caract = jugar_caract.reset_index(drop=True)
jugar_caract = jugar_caract.drop([11], axis = 1)

In [55]:
jugar_caract.head(2)

,Valoración equipo actual,Mejor valoración,Ataque,Regate,Aceleración,Potencia tiro,Agresividad,Defensa,Portería
0,76,72,54,47,62,35,65,67,10
1,83,70,59,80,84,65,34,25,7


In [52]:
nuevas_columnas = {
    2 : 'Valoración equipo actual',
    3: 'Mejor valoración',
    4: "Ataque", 
    5: "Regate", 
    6: "Aceleración",
    7: 'Potencia tiro',
    8: 'Agresividad',
    9: 'Defensa',
    10: 'Portería'}

In [53]:
jugar_caract.rename(columns = nuevas_columnas, inplace = True)
jugar_caract.head()

,Valoración equipo actual,Mejor valoración,Ataque,Regate,Aceleración,Potencia tiro,Agresividad,Defensa,Portería
0,76,72,54,47,62,35,65,67,10
1,83,70,59,80,84,65,34,25,7
2,70,72,59,72,79,70,60,40,6
3,74,72,46,67,71,75,64,30,12
4,74,72,58,71,73,73,74,19,5


In [54]:
jugar_caract.to_csv('../../data/precio/valoraciones_jugadores.csv')